In [1]:
import os, json, gzip, pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
with open("../dataset/cyberlab_2020-02-29.json") as f:
    data = json.load(f)

In [3]:
events = []
for session in data:
    (session_id, session_events), = session.items()
    for e in session_events:
        e["session_id"] = session_id
        events.append(e)

df = pd.DataFrame(events)
print(df.shape)
df.head()

(379033, 36)


,session_id,dst_host_identifier,eventid,timestamp,src_ip_identifier,dst_ip_identifier,message,protocol,src_port,sensor,...,type,outfile,destfile,duplicate,shasum,url,ttylog,size,filename,data
0,0258c1b64b66,3c84036c67663a493ac99f838e65b00f36c939d90c7814...,cowrie.session.connect,2020-02-29T00:06:51.433806Z,f0bb16e4a6ed33e675e3fdea10ee11afc7f5a9bc57406c...,None,New connection: f0bb16e4a6ed33e675e3fdea10ee11...,ssh,53566.0,ubuntu_basic_pool,...,None,None,None,None,None,None,None,NaN,None,None
1,0258c1b64b66,3c84036c67663a493ac99f838e65b00f36c939d90c7814...,cowrie.client.version,2020-02-29T00:06:51.457121Z,f0bb16e4a6ed33e675e3fdea10ee11afc7f5a9bc57406c...,None,Remote SSH version: b'SSH-2.0-libssh-0.6.3',None,NaN,ubuntu_basic_pool,...,None,None,None,None,None,None,None,NaN,None,None
2,0258c1b64b66,3c84036c67663a493ac99f838e65b00f36c939d90c7814...,cowrie.client.kex,2020-02-29T00:06:51.478455Z,f0bb16e4a6ed33e675e3fdea10ee11afc7f5a9bc57406c...,None,SSH client hassh fingerprint: 51cba57125523ce4...,None,NaN,ubuntu_basic_pool,...,None,None,None,None,None,None,None,NaN,None,None
3,0258c1b64b66,3c84036c67663a493ac99f838e65b00f36c939d90c7814...,cowrie.login.failed,2020-02-29T00:06:51.663160Z,f0bb16e4a6ed33e675e3fdea10ee11afc7f5a9bc57406c...,None,login attempt [root/147258] failed,None,NaN,ubuntu_basic_pool,...,None,None,None,None,None,None,None,NaN,None,None
4,0258c1b64b66,3c84036c67663a493ac99f838e65b00f36c939d90c7814...,cowrie.session.closed,2020-02-29T00:06:52.686702Z,f0bb16e4a6ed33e675e3fdea10ee11afc7f5a9bc57406c...,None,Connection lost after 1 seconds,None,NaN,ubuntu_basic_pool,...,None,None,None,None,None,None,None,NaN,None,None


In [4]:
df.describe(include="all")

,session_id,dst_host_identifier,eventid,timestamp,src_ip_identifier,dst_ip_identifier,message,protocol,src_port,sensor,...,type,outfile,destfile,duplicate,shasum,url,ttylog,size,filename,data
count,379033,379033,379033,379033,379033,0,379033,71172,73292.000000,379033,...,119,0,0,12201,12201,0,12201,12201.000000,0,0
unique,71172,24,10,379033,2783,0,89128,1,NaN,5,...,1,0,0,1,1,0,1,NaN,0,0
top,17ad4067463e,912f523c81eaa22b69e4e40dcd38ff6a50e50a45d2cb13...,cowrie.login.failed,2020-02-29T00:06:51.433806Z,374d2a0a69375cd160fba8a3acd47c6b88c8f819f65405...,NaN,Remote SSH version: b'SSH-2.0-libssh-0.6.3',ssh,NaN,ubuntu_basic_pool,...,ssh-rsa,NaN,NaN,True,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,NaN,var/lib/cowrie/tty/e3b0c44298fc1c149afbf4c8996...,NaN,NaN,NaN
freq,63,32697,80970,1,17868,NaN,38948,71172,NaN,304581,...,119,NaN,NaN,12201,12201,NaN,12201,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43037.859343,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.027539,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13565.286210,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.710768,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36112.750000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44834.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53400.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.000000,NaN,NaN


ta bort dst_ip_identifier? 0 värden
timestamp --> alla är unika, inga olika tider


In [ ]:
for col in df.columns:
    print(col)

session_id
dst_host_identifier
eventid
timestamp
src_ip_identifier
dst_ip_identifier
message
protocol
src_port
sensor
geolocation_data
arch
duration
ssh_client_version
username
password
hasshAlgorithms
macCS
langCS
compCS
encCS
hassh
kexAlgs
keyAlgs
fingerprint
key
type
outfile
destfile
duplicate
shasum
url
ttylog
size
filename
data


In [12]:
print(df['duration'].head)

<bound method NDFrame.head of 0              NaN
1              NaN
2              NaN
3              NaN
4         1.123112
            ...   
379028         NaN
379029         NaN
379030         NaN
379031         NaN
379032    1.656798
Name: duration, Length: 379033, dtype: float64>


In [ ]:
df.isna().sum()

session_id                  0
dst_host_identifier         0
eventid                     0
timestamp                   0
src_ip_identifier           0
dst_ip_identifier      379033
message                     0
protocol               307861
src_port               305741
sensor                      0
geolocation_data            0
arch                   379033
duration               299219
ssh_client_version     314428
username               293475
password               293594
hasshAlgorithms        317589
macCS                  317589
langCS                 317589
compCS                 317589
encCS                  317589
hassh                  317589
kexAlgs                317589
keyAlgs                317589
fingerprint            378914
key                    378914
type                   378914
outfile                379033
destfile               379033
duplicate              366832
shasum                 366832
url                    379033
ttylog                 366832
size      

eventid, timestamp, message, sensor, src_ip_identifier --> inga nan värden
dst_ip_identifier, arch, filename, data --> ta bort

In [ ]:
# Remove columns with too many missing values
# <30.000 missing values and irrelevant columns for the analysis 

df = df.drop(columns=["dst_ip_identifier", "arch", "hasshAlgorithms", "macCS", "langCS", "compCS", 
    "encCS", "hassh", "kexAlgs", "keyAlgs", "fingerprint", "key", "type",
    "outfile", "destfile", "duplicate", "shasum", "url", "ttylog", "size",
    "filename", "data"])

In [7]:
print(df.columns.tolist())
print(df.shape)

['session_id', 'dst_host_identifier', 'eventid', 'timestamp', 'src_ip_identifier', 'message', 'protocol', 'src_port', 'sensor', 'geolocation_data', 'duration', 'ssh_client_version', 'username', 'password']
(379033, 14)
